## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [2]:
df = pd.read_csv('Multifamily Projects \'23 - Sheet1.csv')

## Image URL collector

In [3]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [4]:
df.columns

Index(['Address', 'Project name',
       'Expected completion (has to be this year by the end of the year)',
       'Size: Number of stories and units', 'Developer', 'Link'],
      dtype='object')

In [5]:
links = []
for url in df['Link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except Exception as e:
        links.append('no_image')
        print(e)

links.append('trd-logo-16x9.jpeg')
links.append('trd-logo-16x9.jpeg')

df['image_links'] = links

In [6]:
links

['https://s14.therealdeal.com/trd/m/up/2021/10/Main-MainImage_45m-loan-705x439.jpg',
 'https://s14.therealdeal.com/trd/m/up/2021/10/lmain-issette-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2021/02/1200-Dev-Motwani-partner-score-56M-construction-loan-for-Dania-Beach-mixed-use-project-v2-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2020/09/soleste-1-650x405.jpg',
 'https://s11.therealdeal.com/trd/m/up/2022/12/main-MIA-Royal-Palm-scores-77M-construction-loan-for-Dania-Beach-project-705x439.jpg',
 'https://s11.therealdeal.com/trd/m/up/2021/05/merrimac-ventures-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2021/06/1200-American-Land-Ventures-Isram-Realty-score-85M-construction-loan-for-Boynton-Beach-apartments-705x439.jpg',
 'trd-logo-16x9.jpeg',
 'trd-logo-16x9.jpeg']

https://s14.therealdeal.com/trd/m/up/2021/10/main-MIA-Location-Ventures-co-living_10.13.21-705x439.png

## Article link formatter

In [7]:
df.dtypes

Address                                                             object
Project name                                                        object
Expected completion (has to be this year by the end of the year)    object
Size: Number of stories and units                                   object
Developer                                                           object
Link                                                                object
image_links                                                         object
dtype: object

In [8]:
df["short_description"] = df["Developer"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Developer'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2021/10...
1    <a href="https://therealdeal.com/miami/2021/10...
2    <a href="https://therealdeal.com/miami/2021/02...
3    <a href="https://therealdeal.com/miami/2020/09...
4    <a href="https://therealdeal.com/miami/2022/12...
5    <a href="https://therealdeal.com/miami/2021/05...
6    <a href="https://therealdeal.com/miami/2021/06...
7    <a href="https://therealdeal.com/miami/tag/13t...
8    <a href="https://therealdeal.com/miami/tag/alt...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [9]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [10]:
df['geo_address'] = df['Address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [11]:
df.columns

Index(['Address', 'Project name',
       'Expected completion (has to be this year by the end of the year)',
       'Size: Number of stories and units', 'Developer', 'Link', 'image_links',
       'short_description', 'remaining_desc', 'description_link',
       'geo_address', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
df

,Address,Project name,Expected completion (has to be this year by the end of the year),Size: Number of stories and units,Developer,Link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"1190 NE 3rd Ave, Pompano Beach, FL 33060",Old Town Square,August/September of this year,10-story project with 281 multifamily units pl...,Cavache Properties and Wilson Enterprises,https://therealdeal.com/miami/2021/10/12/pompa...,https://s14.therealdeal.com/trd/m/up/2021/10/M...,Cavache Properties,and Wilson Enterprises,"<a href=""https://therealdeal.com/miami/2021/10...","1190 NE 3rd Ave, Pompano Beach, FL 33060","(1190 NE 3rd Ave, Pompano Beach, FL 33060, USA...","(26.2429769, -80.1207117, 0.0)",26.242977,-80.120712,0.0
1,1625 Northwest 20th Street in Miami's Allapatt...,The Julia,this year,"14-story, 323-unit project","Lissette Calderon, through her Neology Life De...",https://therealdeal.com/miami/2021/10/13/lisse...,https://s14.therealdeal.com/trd/m/up/2021/10/l...,"Lissette Calderon,",through her Neology Life Development Group,"<a href=""https://therealdeal.com/miami/2021/10...",1625 Northwest 20th Street in Miami's Allapatt...,"(1625 NW 20th St, Miami, FL 33142, USA, (25.79...","(25.7952228, -80.2221551, 0.0)",25.795223,-80.222155,0.0
2,4 North Federal Highway in Dania Beach,Soleste Cityline,February 2023,8-story building with 340 units,"Dev Motwani, of Merrimac Ventures, and Estate ...",https://therealdeal.com/miami/2021/02/12/dev-m...,https://s12.therealdeal.com/trd/m/up/2021/02/1...,"Dev Motwani, of","Merrimac Ventures, and Estate Companies","<a href=""https://therealdeal.com/miami/2021/02...",4 North Federal Highway in Dania Beach,"(4 N Federal Hwy, Dania Beach, FL 33004, USA, ...","(26.05253, -80.14326799999999, 0.0)",26.052530,-80.143268,0.0
3,16395 Biscayne Boulevard,Soleste NoMi Beach,2023,23-story building with 367 units,Estate Companies,https://therealdeal.com/miami/2020/09/17/estat...,https://s12.therealdeal.com/trd/m/up/2020/09/s...,Estate Companies,,"<a href=""https://therealdeal.com/miami/2020/09...",16395 Biscayne Boulevard,"(16395 Biscayne Blvd, North Miami Beach, FL 33...","(25.9285672, -80.1520188, 0.0)",25.928567,-80.152019,0.0
4,600 East Dania Beach Boulevard,Elevate Apartments,Early 2024,293 units,"Dan Kodsi's Royal Palm Companies, in partnersh...",https://therealdeal.com/miami/2022/12/06/kodsi...,https://s11.therealdeal.com/trd/m/up/2022/12/m...,Dan Kodsi's Royal,"Palm Companies, in partnership with developer...","<a href=""https://therealdeal.com/miami/2022/12...",600 East Dania Beach Boulevard,"(600 E Dania Beach Blvd, Dania Beach, FL 33004...","(26.0515467, -80.13397049999999, 0.0)",26.051547,-80.133970,0.0
5,1711 North University Drive in Plantation,Merrimac Plantation,2023,306 units,Motwani family's Merrimac Ventures,https://therealdeal.com/miami/2021/05/27/motwa...,https://s11.therealdeal.com/trd/m/up/2021/05/m...,Motwani family's,Merrimac Ventures,"<a href=""https://therealdeal.com/miami/2021/05...",1711 North University Drive in Plantation,"(1711 N University Dr, Plantation, FL 33322, U...","(26.1461595, -80.2578431, 0.0)",26.146159,-80.257843,0.0
6,630 East Woolbright Road in Boynton Beach,NaN,2023,"10 stories, 319 apartments","American Land Ventures, Isram Realty Group",https://therealdeal.com/miami/2021/06/01/ameri...,https://s12.therealdeal.com/trd/m/up/2021/06/1...,American Land,"Ventures, Isram Realty Group","<a href=""https://therealdeal.com/miami/2021/06...",630 East Woolbright Road in Boynton Beach,"(630 E Woolbright Rd, Boynton Beach, FL 33435,...","(26.5141215, -80.05648289999999, 0.0)",26.514122,-80.056483,0.0
7,3050 SW 37th Avenue at the Link at Douglas pro...,Cascade,2023,"37 stories, 421 units",13th Floor Investments,https://therealdeal.com/miami/tag/13th-floor-i...,trd-logo-16x9.jpeg,13th Floor,Investments,"<a href=""https://therealdeal.com/miami/tag/13t...",3050 SW 37th Avenue at the Link at Douglas pro...,"(3060 SW 37th Ct, Miami, FL 331

In [13]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [14]:
df

,Address,Project name,Expected completion (has to be this year by the end of the year),Size: Number of stories and units,Developer,Link,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"1190 NE 3rd Ave, Pompano Beach, FL 33060",Old Town Square,August/September of this year,10-story project with 281 multifamily units pl...,Cavache Properties and Wilson Enterprises,https://therealdeal.com/miami/2021/10/12/pompa...,https://s14.therealdeal.com/trd/m/up/2021/10/M...,Cavache Properties,and Wilson Enterprises,"<a href=""https://therealdeal.com/miami/2021/10...","1190 NE 3rd Ave, Pompano Beach, FL 33060","(1190 NE 3rd Ave, Pompano Beach, FL 33060, USA...","(26.2429769, -80.1207117, 0.0)",26.242977,-80.120712,0.0
1,1625 Northwest 20th Street in Miami's Allapatt...,The Julia,this year,"14-story, 323-unit project","Lissette Calderon, through her Neology Life De...",https://therealdeal.com/miami/2021/10/13/lisse...,https://s14.therealdeal.com/trd/m/up/2021/10/l...,"Lissette Calderon,",through her Neology Life Development Group,"<a href=""https://therealdeal.com/miami/2021/10...",1625 Northwest 20th Street in Miami's Allapatt...,"(1625 NW 20th St, Miami, FL 33142, USA, (25.79...","(25.7952228, -80.2221551, 0.0)",25.795223,-80.222155,0.0
2,4 North Federal Highway in Dania Beach,Soleste Cityline,February 2023,8-story building with 340 units,"Dev Motwani, of Merrimac Ventures, and Estate ...",https://therealdeal.com/miami/2021/02/12/dev-m...,https://s12.therealdeal.com/trd/m/up/2021/02/1...,"Dev Motwani, of","Merrimac Ventures, and Estate Companies","<a href=""https://therealdeal.com/miami/2021/02...",4 North Federal Highway in Dania Beach,"(4 N Federal Hwy, Dania Beach, FL 33004, USA, ...","(26.05253, -80.14326799999999, 0.0)",26.052530,-80.143268,0.0
3,16395 Biscayne Boulevard,Soleste NoMi Beach,2023,23-story building with 367 units,Estate Companies,https://therealdeal.com/miami/2020/09/17/estat...,https://s12.therealdeal.com/trd/m/up/2020/09/s...,Estate Companies,,"<a href=""https://therealdeal.com/miami/2020/09...",16395 Biscayne Boulevard,"(16395 Biscayne Blvd, North Miami Beach, FL 33...","(25.9285672, -80.1520188, 0.0)",25.928567,-80.152019,0.0
4,600 East Dania Beach Boulevard,Elevate Apartments,Early 2024,293 units,"Dan Kodsi's Royal Palm Companies, in partnersh...",https://therealdeal.com/miami/2022/12/06/kodsi...,https://s11.therealdeal.com/trd/m/up/2022/12/m...,Dan Kodsi's Royal,"Palm Companies, in partnership with developer...","<a href=""https://therealdeal.com/miami/2022/12...",600 East Dania Beach Boulevard,"(600 E Dania Beach Blvd, Dania Beach, FL 33004...","(26.0515467, -80.13397049999999, 0.0)",26.051547,-80.133970,0.0
5,1711 North University Drive in Plantation,Merrimac Plantation,2023,306 units,Motwani family's Merrimac Ventures,https://therealdeal.com/miami/2021/05/27/motwa...,https://s11.therealdeal.com/trd/m/up/2021/05/m...,Motwani family's,Merrimac Ventures,"<a href=""https://therealdeal.com/miami/2021/05...",1711 North University Drive in Plantation,"(1711 N University Dr, Plantation, FL 33322, U...","(26.1461595, -80.2578431, 0.0)",26.146159,-80.257843,0.0
6,630 East Woolbright Road in Boynton Beach,NaN,2023,"10 stories, 319 apartments","American Land Ventures, Isram Realty Group",https://therealdeal.com/miami/2021/06/01/ameri...,https://s12.therealdeal.com/trd/m/up/2021/06/1...,American Land,"Ventures, Isram Realty Group","<a href=""https://therealdeal.com/miami/2021/06...",630 East Woolbright Road in Boynton Beach,"(630 E Woolbright Rd, Boynton Beach, FL 33435,...","(26.5141215, -80.05648289999999, 0.0)",26.514122,-80.056483,0.0
7,3050 SW 37th Avenue at the Link at Douglas pro...,Cascade,2023,"37 stories, 421 units",13th Floor Investments,https://therealdeal.com/miami/tag/13th-floor-i...,trd-logo-16x9.jpeg,13th Floor,Investments,"<a href=""https://therealdeal.com/miami/tag/13t...",3050 SW 37th Avenue at the Link at Douglas pro...,"(3060 SW 37th Ct, Miami, FL 331

In [15]:
df['description_link'].iloc[0]

'<a href="https://therealdeal.com/miami/2021/10/12/pompano-beach-mixed-use-project-scores-45m-construction-loan/" target="_blank" rel="noopener noreferrer">Cavache Properties</a> and Wilson Enterprises'

In [16]:
def popup_html(row):
    i = row
    address = df['Address'].iloc[i]
    description_link = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    size = df['Size: Number of stories and units'].iloc[i]
    expected_completion = df['Expected completion (has to be this year by the end of the year)'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developers: <strong>{}'''.format(description_link) + '''</strong><br>
    <strong>{}'''.format(size) + '''</strong><br>
    Expected Completion: <strong>{}'''.format(expected_completion) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [17]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [18]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2021/10/12/pompano-beach-mixed-use-project-scores-45m-construction-loan/" target="_blank" rel="noopener noreferrer">Cavache Properties</a> and Wilson Enterprises'

In [19]:
import folium
import branca

# f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Multifamily Projects '23")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
# m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [21]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [22]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/multi_family_projects_2023
